In [1]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
columns = ['text', 'label']

In [4]:
DATA_PATH = Path('../data/')
LABEL_PATH = Path('../labels/')

In [5]:
ivr_df = pd.read_excel(DATA_PATH/'centrica_master_tagging_sheet.xlsx', sheet_name=2, index_col="row")

In [6]:
ivr_df['app_tag'].nunique()

264

In [7]:
ivr_df.head(10)

,occurrence,transcription,action,object,sem_tag,app_tag,dummy_utt,occ_partition_train,occ_partition_eval,occ_treatment_PG_GENERAL_TREATMENT,occ_treatment_EF_ENERGY_SOLUTIONS_TREATMENT,occ_treatment_CO_EARLY_DEBT_TREATMENT,occ_treatment_BGSM_SMART_ENQUIRIES_TREATMENT,occ_treatment_BGS_BREAKDOWN_P1_TREATMENT,occ_treatment_BGB_GENERAL_ENQUIRIES_TREATMENT,occ_treatment_BGS_ACQUISITIONS_1_MARCOMMS_1_DIVERT_TREATMENT,occ_treatment_SAINSBURYS_ENERGY_SOLUTIONS_TREATMENT,occ_treatment_BGS_HIVE_SUPPORT_TREATMENT
row,,,,,,,,,,,,,,,,,,
0,1,enquiry my my bill,enquire,bill,enquire-bill,enquire-bill,NaN,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,my hot water system has gone up the street,problem,hot_water,problem-hot_water,problem-hot_water,NaN,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
2,1,about the homecare agreement,vague,homecare,vague-homecare,enquire-homecare,NaN,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,1,annual landlord gas safety,enquire,safety_certificate,enquire-safety_certificate,enquire-safety_certificate,NaN,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
4,1,to find out what time the engineer is coming,enquire,engineer_when,enquire-engineer_when,enquire-appointment_when,NaN,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
6,1,cylinder replacement,vague,heating,vague-heating,enquire-heating,NaN,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
7,14,new gas card,request,card,request-card,request-card,NaN,13,1.0,9.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,1,i just want to have general enquiries about,enquire,vague,enquire-vague,enquire-vague,NaN,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
10,1,the previous tariff is the housing authority actually because they use it,vague,tariff,vague-tariff,enquire-tariff,NaN,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [8]:
ivr_df = ivr_df[['transcription', 'app_tag']]

In [9]:
ivr_df.columns = columns

In [10]:
web_chat_df = pd.read_csv(DATA_PATH/'labelled_so_far.csv', header=None)

In [11]:
web_chat_df.columns = columns

In [12]:
web_chat_df.shape

(4929, 2)

In [13]:
web_chat_df['label'].nunique()

253

In [14]:
# intent_df = pd.concat([ivr_df, web_chat_df], axis=0).reset_index(drop=True)
intent_df = ivr_df
intent_df.shape

(27069, 2)

In [15]:
mapped_labels = pd.read_csv(LABEL_PATH/'mapped_labels.csv')
mapped_labels.head()

,label,mapped_label
0,cancel-account,cancel-energy
1,cancel-agreement,cancel-contract
2,cancel-appointment,cancel-appointment
3,cancel-appointment_h,cancel-appointment
4,cancel-appointment_s,cancel-appointment


In [16]:
intent_df = pd.merge(intent_df, mapped_labels, how='left', on='label' )

In [17]:
intent_df.columns = ['text', 'old_label', 'label']

In [18]:
intents = intent_df.groupby('label')['text'].count()
intents = intents.sort_values(ascending=True).reset_index()
intents.columns = ['label', 'count']

In [23]:
web_chat_df = pd.merge(web_chat_df, mapped_labels, how='left', on='label' )
web_chat_df.columns = ['text', 'old_label', 'label']

In [19]:
intents_less_utterances = intents[intents['count'] < 10]

In [20]:
intents_less_utterances

,label,count
0,vague-compensation,1
1,enquire-fuel_direct,1
2,enquire-emergency_credit,1
3,enquire-insulation,2
4,vague-pest_control,2
5,enquire-authority,2
6,change-payment_plan,2
7,problem-payment_plan,2
8,report-gas_emergency,2
9,enquire-security_deposit,2


In [24]:
low_occurances = intent_df[intent_df['label'].isin(intents_less_utterances['label'])]
web_chat_df_low_utterances = web_chat_df[web_chat_df['label'].isin(intents_less_utterances['label'])]


In [25]:
LOW_DATA_PATH = DATA_PATH/'low_occurances'
LOW_DATA_PATH.mkdir(exist_ok=True)

In [26]:
low_occurances.shape, web_chat_df_low_utterances.shape

((310, 3), (392, 3))

In [32]:
def create_utterances_low_label(label):
    low_occurances_orig = low_occurances[low_occurances['label'] == label]
    low_occurances_orig['source'] = 'ivr' 
    low_occurances_supporting = web_chat_df_low_utterances[web_chat_df_low_utterances['label'] == label]
    low_occurances_supporting['source'] = 'webchat'
    pd.concat([low_occurances_orig, low_occurances_supporting], axis=0).to_csv(str(LOW_DATA_PATH/'{}.csv'.format(label)))

In [35]:
web_chat_df_low_utterances[web_chat_df_low_utterances['label'] == 'enquire-meter_reset']

,text,old_label,label


In [38]:
intent_df[intent_df['label'] == 'change-payment']

,text,old_label,label
483,to adjust my payments,change-payment,change-payment
572,i would like to change the monthly payment,change-payment,change-payment
587,to increase my payment,change-payment,change-payment
1460,change payment method,change-payment,change-payment
1638,changing my payment date,change-payment,change-payment
1662,i need to change payment details,change-payment,change-payment
2039,call to correct my payment,change-payment,change-payment
2071,change monthly payment,change-payment,change-payment
2865,change payment details,change-payment,change-payment
3504,i want to increase my payments,change-payment,change-payment


In [60]:
web_chat_df_low_utterances.shape

(392, 3)

In [41]:
all_low_utterances = pd.concat([web_chat_df_low_utterances, low_occurances], axis=0)

In [43]:
all_low_utterances.sort_values('label', inplace=True)

In [48]:
unique_low_labels = list(all_low_utterances['label'].unique())

In [50]:
all_low_utterances[all_low_utterances['label']==unique_low_labels[1]]

,text,old_label,label
5011,to cancel a direct_debit,cancel-direct_debit,cancel-direct_debit
4934,to cancel my direct_debit,cancel-direct_debit,cancel-direct_debit
4426,stop direct_debit,cancel-direct_debit,cancel-direct_debit
10341,direct_debit cancellation,cancel-direct_debit,cancel-direct_debit
3663,cancel direct_debit,cancel-direct_debit,cancel-direct_debit
1332,to cancel direct_debit,cancel-direct_debit,cancel-direct_debit
970,i would like to cancel my direct_debit,cancel-direct_debit,cancel-direct_debit
1584,hi. I have just switched online to XXX fixed tariff but I dont want to pay by direct debit. How can I make sure this is done.(12 Valverde House. Eaton Gardens. BN3 3TU) Sandra Davis - 21.6.XXX,cancel-direct_debit,cancel-direct_debit
1557,Hi Mohammed Im just looking to change the date that the direct debit gets taken. It currently goes on the 20th but Id like it to go on the 25th?,cancel-direct_debit,cancel-direct_debit
22851,cancelling direct_debit,cancel-direct_debit,cancel-direct_debit


In [51]:
all_low_utterances.to_csv(str(DATA_PATH/'low_utterances.csv'))

In [21]:
labels = pd.DataFrame(intent_df['label'].unique())

In [22]:
exclude_intents = ['vague-vague', 'None', 'delete-delete', 'garbage-garbage']

In [23]:
intent_cleaned_df = intent_df[intent_df['label'].isin(exclude_intents) == False]
intent_cleaned_df = intent_cleaned_df[intent_cleaned_df['label'].isna() == False]

In [24]:
intent_cleaned_df.shape

(16048, 3)

In [25]:
try:
    del intent_cleaned_df['old_label']
except:
    pass

In [26]:
excluded_intents = intent_df[intent_df['label'].isin(exclude_intents)]

In [27]:
excluded_intents.shape

(8377, 3)

In [28]:
intent_cleaned_df.to_csv(str(DATA_PATH/'consolidated_intents.csv'), index=None)

In [29]:
labels.columns = ['label']

In [30]:
# labels.to_csv('../labels/labels.csv', index=None, header=None)

In [31]:
train_df, val_df = train_test_split(intent_cleaned_df, test_size=0.2, shuffle=True)

In [32]:
train_df.to_csv(str(DATA_PATH/'train_4.csv'), index=None)
val_df.to_csv(str(DATA_PATH/'val_4.csv'), index=None)

In [33]:
train_df.shape, val_df.shape

((12838, 2), (3210, 2))

In [ ]:
train_es = pd.read_csv(DATA_PATH/'data_augmentation'/'train_es.csv')

In [ ]:
train_es.dropna(inplace=True)
train_es.to_csv(str(DATA_PATH/'data_augmentation'/'train_es.csv'), index=None)

In [ ]:
train_es

In [ ]:
train_de = pd.read_csv(DATA_PATH/'data_augmentation'/'train_de.csv')
train_de.dropna(inplace=True)
train_de.to_csv(str(DATA_PATH/'data_augmentation'/'train_de.csv'), index=None)

In [ ]:
train_de.shape

In [ ]:
train_fr = pd.read_csv(DATA_PATH/'data_augmentation'/'train_fr.csv')
train_fr.dropna(inplace=True)
train_fr.to_csv(str(DATA_PATH/'data_augmentation'/'train_fr.csv'), index=None)

In [ ]:
train_fr

In [ ]:
train_augmented = pd.concat([train_df, train_de, train_fr]).sample(frac=1).reset_index(drop=True)

In [ ]:
train_augmented = train_augmented.dropna()

In [ ]:
train_augmented.shape

In [ ]:
train_augmented.to_csv(str(DATA_PATH/'train_aug.csv'), index=False)

In [34]:
change_phone_texts = [
    "set phone number to 02091122212", 
    "change phone to 07892238872", 
    "set customer's phone to 03001292212", 
    "change customer's phone", 
    "set phone number", 
    "set mobile number to 075512211122", 
    "change phone", 
    "change customer's phone number", 
    "update phone number", 
    "change customer's landline", 
    "change customer's mobile number", 
    "update mobile number to 077891123763", 
    "update landline to 011892234432", 
    "please add telephone number", 
    "please add customers telephone number", 
    "update telephone 07932212233", 
    "add mobile", 
    "add phone number",
    "can you update our phone number to 07508443732",
    "change customer landline to 02078832232",
    "change customer's phone number to 07509332348",
    "change phone number to 03772121212",
    "change telephone number to 079891233321",
    "here is our new landline number . its 011892212121",
    "hi my mobile number is 07932 033764.",
    "hi my mobile number is 07932033764.",
    "hi you have our old telephone number our new one is 01934645678",
    "how do i change the phone number for jack ?",
    "my new landline is 011893322323",
    "my new mobile number is 07428322121",
    "change customer's phone number to 07509332348",
    "change telephone number to 079891233321",
    "here is our new landline number . its 011892212121",
    "set phone number to 02091122212", 
    "change phone to 07892238872"
]

labelled_values = [{ "text":  text, "label": 'change-phone'} for text in change_phone_texts]

In [35]:
change_phone_df = pd.DataFrame(labelled_values)
change_phone_df = change_phone_df[['text', 'label']]

In [36]:
change_phone_df.shape

(35, 2)

In [37]:
change_phone_df = change_phone_df.sample(frac=1)

In [38]:
change_phone_df.to_csv(str(DATA_PATH/'change-phone.csv'), index=None)

In [39]:
train_phone_df = pd.concat([train_df, change_phone_df]).sample(frac=1)

In [40]:
train_phone_df.to_csv(str(DATA_PATH/'train_4.csv'), index=None)

In [ ]:
train_phone_df.head()

In [ ]:
train_old = pd.read_csv(DATA_PATH/'val.csv')

In [ ]:
train_old.shape

In [ ]:
# intent_df = pd.merge(train_old, mapped_labels, how='left', on='label' )

In [ ]:
# intent_df.columns = ['text', 'old_label', 'label']

In [ ]:
# try:
#     del intent_df['old_label']
# except:
#     pass

In [ ]:
# intent_df.to_csv(str(DATA_PATH/'val.csv'), index=None)

In [ ]:
list(train_old.apply(process_data, axis=1))[0].labels